# Chapter 7. 통계학과 머신러닝

## 7.4 선형모델과 신경망

### 7.4.10 분석준비

In [2]:
# 수치 계산에 사용하는 라이브러리
import numpy as np
import pandas as pd
import scipy as sp

# 통계모델을 추정하는 데 사용하는 라이브러리
import statsmodels.formula.api as smf
import statsmodels.api as sm

# 다층 퍼셉트론용 애플리케이션
from sklearn.neural_network import MLPClassifier

# 샘플 데이터 불러오기
from sklearn.datasets import load_iris

# 테스트 데이터와 훈련 데이터 분리
from sklearn.model_selection import train_test_split

# 데이터 표준화용 라이브러리
from sklearn.preprocessing import StandardScaler

# 표시자릿수 지정
%precision 3

'%.3f'

### 7.4.11 데이터 읽기와 다듬기

In [3]:
# 데이터 불러들이기
iris = load_iris()

In [4]:
# 독립변수 이름
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [5]:
# 종속변수 이름
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [6]:
# 독립변수 sepal(꽃받침)만 가져옵니다
X = iris.data[50:150, 0:2]
# 붓꽃의 종류도 2개만 가져옵니다
y = iris.target[50:150]

print("독립변수 행 수, 열 수：", X.shape)
print("종속변수 행 수, 열 수：", y.shape)

독립변수 행 수, 열 수： (100, 2)
종속변수 행 수, 열 수： (100,)


In [7]:
# 데이터를 훈련데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state = 2)

print("독립변수 행 수, 열 수：", X_train.shape)
print("종속변수 행 수, 열 수：", y_train.shape)

독립변수 행 수, 열 수： (75, 2)
종속변수 행 수, 열 수： (75,)


### 7.4.12 로지스틱 회귀

In [8]:
# 종속변수
y_train[0:10]

array([1, 1, 2, 2, 2, 2, 1, 1, 1, 1])

In [9]:
# 데이터 정리
# 독립변수 데이터프레임
X_train_df = pd.DataFrame(
    X_train, columns = ["sepal_len", "sepal_wid"])
# 종속변수 데이터프레임
y_train_df = pd.DataFrame({"species": y_train - 1})
# 데이터프레임 결합
iris_train_df = pd.concat(
    [y_train_df, X_train_df], axis=1)
# 결과 출력
print(iris_train_df.head(3))

   species  sepal_len  sepal_wid
0        0        5.7        2.8
1        0        6.6        3.0
2        1        6.1        3.0


In [10]:
# 모델로 만들기
# 모든 변수를 넣은 모델
logi_mod_full = smf.glm(
    "species ~ sepal_len + sepal_wid", data = iris_train_df,
    family=sm.families.Binomial()).fit()

# 길이만
logi_mod_len = smf.glm(
    "species ~ sepal_len", data = iris_train_df,
    family=sm.families.Binomial()).fit()

# 폭만
logi_mod_wid = smf.glm(
    "species ~ sepal_wid", data = iris_train_df,
    family=sm.families.Binomial()).fit()

# Null모델
logi_mod_null = smf.glm(
    "species ~ 1", data = iris_train_df,
    family=sm.families.Binomial()).fit()

# AIC비교
print("full", logi_mod_full.aic.round(3))
print("len ", logi_mod_len.aic.round(3))
print("wid ", logi_mod_wid.aic.round(3))
print("null", logi_mod_null.aic.round(3))

full 76.813
len  76.234
wid  92.768
null 105.318


In [11]:
# 추정된 계수
logi_mod_len.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-16.4152,4.000,-4.104,0.000,-24.256,-8.575
sepal_len,2.6478,0.639,4.142,0.000,1.395,3.901


In [12]:
# 데이터 정리
X_test_df = pd.DataFrame(
    X_test, columns = ["sepal_len", "sepal_wid"])

# 피팅 예측
logi_fit = logi_mod_len.fittedvalues.round(0)
logi_pred = logi_mod_len.predict(X_test_df).round(0)

# 정답 수
true_train = sp.sum(logi_fit == (y_train - 1))
true_test = sp.sum(logi_pred == (y_test - 1))

# 적중률
result_train = true_train / len(y_train)
result_test = true_test / len(y_test)

# 결과출력
print("訓練データの的中率　：", result_train)
print("テストデータの的中率：", result_test)

訓練データの的中率　： 0.7466666666666667
テストデータの的中率： 0.68


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: scipy.sum is deprecated and will be removed in SciPy 2.0.0, use numpy.sum instead
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: scipy.sum is deprecated and will be removed in SciPy 2.0.0, use numpy.sum instead
  # This is added back by InteractiveShellApp.init_path()


### 7.4.13 표준화

In [13]:
# 표준화를 위한 준비
scaler = StandardScaler()
scaler.fit(X_train)
# 표준화
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
sp.std(X_train_scaled, axis=0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: scipy.std is deprecated and will be removed in SciPy 2.0.0, use numpy.std instead
  """Entry point for launching an IPython kernel.


array([1., 1.])

In [27]:
sp.std(X_test_scaled, axis=0)

array([ 0.74 ,  0.679])

### 7.4.14 신경망

In [16]:
nnet = MLPClassifier(
    hidden_layer_sizes = (100,100),
    alpha = 0.07,
    max_iter = 10000,
    random_state = 0)
nnet.fit(X_train_scaled, y_train)

# 정답률
print("훈련데이터 적중률　：", nnet.score(X_train_scaled, y_train))
print("테스트데이터 적중률：", nnet.score(X_test_scaled, y_test))

훈련데이터 적중률　： 0.8933333333333333
테스트데이터 적중률： 0.68
